In [1]:
import pandas as pd
import numpy as np
import string

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
df = pd.read_csv('resume_data.csv')
df.drop_duplicates(inplace=True)
stop_words = set(stopwords.words('russian'))  # Используйте соответствующий язык
stemmer = SnowballStemmer('russian')  # Используйте соответствующий языкэ
def preprocess_text(text):
    # Приведем к нижнему регистру
    text = text.lower()
    # Удалим пунктуацию
    text = "".join([char for char in text if char not in string.punctuation])
    # Токенизация
    words = word_tokenize(text)
    # Удалим стоп-слова и применим стемминг
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    
    
    return words

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

job_description = """
Системный администратор
Знание и опыт работы с серверными операционными системами Windows Server и Linux.
Умение настраивать сетевое оборудование.
Опыт работы с системами виртуализации и облачными сервисами.
Способность быстро решать проблемы с IT-инфраструктурой.
Навыки работы с базами данных и системами резервного копирования.
"""

text_columns = ['Position', 'Specializations', 'Previous_Positions', 'Languages', 'Education', 'About_Me', 'Skills']
resume_texts = df[text_columns].fillna('').agg(' '.join, axis=1).tolist()
all_texts = [job_description] + resume_texts

In [5]:
from gensim.models import FastText
from gensim.models.fasttext import load_facebook_model
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = load_facebook_model('cc.ru.300.bin')


KeyboardInterrupt: 

In [4]:

# Функция для получения вектора документа путем усреднения векторов слов
def document_vector(doc):
    # Удаляем слова, которых нет в модели
    words = [word for word in doc.split() if word in model.vocab]
    if len(words) == 0:
       return np.zeros(model.vector_size)
    else:
       # Усредняем векторы слов, чтобы получить вектор документа
       return np.mean(model[words], axis=0)

# Преобразуем описание вакансии и резюме в векторы
job_vec = document_vector(job_description)
resume_vecs = np.array([document_vector(doc) for doc in resume_texts])

# Вычисляем косинусное сходство между вакансией и каждым резюме
cosine_similarities = cosine_similarity([job_vec], resume_vecs).flatten()

# Создаем словарь сопоставлений индекс резюме -> сходство
resume_similarities = dict(enumerate(cosine_similarities))

# Сортируем резюме по убыванию сходства
sorted_resume_similarities = sorted(resume_similarities.items(), key=lambda item: item[1], reverse=True)

sorted_resume_similarities

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xba in position 0: invalid start byte